<a href="https://colab.research.google.com/github/et169tkm/gpt35turbo_test_text_parsing/blob/main/ChatGPT_API_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports and functions { vertical-output: true }

!pip install openai >/dev/null

import json
import numpy as np
import os
import openai


API_KEY = input("API Key")
openai.api_key = API_KEY


# functions

# Recursively check if element "e" matches, perform fn() on it if it matches
def perform_on_type(e, matcher, fn):
  if matcher(e):
    fn(e)
  else:
    if getattr(e, "children", None): # check if it has the "children" attribute
      for child in e.children:
        perform_on_type(child, matcher, fn)

In [ ]:
#@title Test data 1 (90. Patcham Court, Brighton Road) { vertical-output: true }

print("""
price: null
floor: 2nd
sqft: null
parking: Undercroft parking
service charge: TBC
ground rent: £35 subject to review and additional rent
chain: No ongoing chain
EPC: D
Council tax: D
Lease: 999 years from 25/12/1972
Tenure: null
""")


# https://www.rightmove.co.uk/properties/130607321?utm_campaign=property-details&utm_content=buying&utm_medium=sharing&utm_source=copytoclipboard#/&channel=RES_BUY
data = """
Key features

    Spacious Two Bedroom Apartment
    Large Reception Room
    EPC Grade D
    Bathroom
    Ensuite Bathroom
    No Ongoing Chain
    Balcony
    Undercroft Parking
    Lease Over 900 Years
    Handy For The Station

Property description

This fantastic two bedroom second floor apartment has a private east facing balcony. The property has many features to boast including gas radiator heating, kitchen, spacious room sizes and undercroft Parking. Other features include an excellent location for Belmont station and easy reach to Sutton Mainline, no ongoing chain. EPC Grade D Lease 999 years from 25/12/1972. Service Charge TBC per annum, ground rent £35 subject to review and additional rent . Council Tax Band D. Lease, ground rent, and maintenance details have been provided by the seller, but their accuracy cannot be guaranteed, as we may not have seen a copy of the original lease. Should you proceed with the purchase of this property, lease details must be verified by your solicitor.

IMPORTANT NOTE TO POTENTIAL PURCHASERS & TENANTS:
We endeavour to make our particulars accurate and reliable, however, they do not constitute or form part of an offer or any contract and none is to be relied upon as statements of representation or fact. The services, systems and appliances listed in this specification have not been tested by us and no guarantee as to their operating ability or efficiency is given. All photographs and measurements have been taken as a guide only and are not precise. Floor plans where included are not to scale and accuracy is not guaranteed. If you require clarification or further information on any points, please contact us, especially if you are traveling some distance to view. POTENTIAL PURCHASERS: Fixtures and fittings other than those mentioned are to be agreed with the seller. POTENTIAL TENANTS: All properties are available for a minimum length of time, with the exception of short term accommodation. Please contact the branch for details. A security deposit of at least one month’s rent is required. Rent is to be paid one month in advance. It is the tenant’s responsibility to insure any personal possessions. Payment of all utilities including water rates or metered supply and Council Tax is the responsibility of the tenant in every case.

SUT220174/2

Reception Room

6.78m x 4.2m

Kitchen

3.63m x 2.2m

Bathroom

2.41m x 1.73m

Ensuite Bathroom

1.98m x 1.73m

Master Bedroom

4.95m x 2.8m

Bedroom

3.6m x 2.41m

Additonal Information

EPC Grade D Lease 999 years from 25/12/1972. Service Charge TBC per annum, ground rent TBC. Council Tax Band D. Lease, ground rent, and maintenance details have been provided by the seller, but their accuracy cannot be guaranteed, as we may not have seen a copy of the original lease. Should you proceed with the purchase of this property, lease details must be verified by your solicitor.
"""

#  Expected answer:
#  {
# 	"price": null,
# 	"has_parking": true,
# 	"service_charge": "TBC", # or null
# 	"ground_rent": "£35 subject to review", # or "Over 900 Years"
# 	"has_chain": false,
# 	"EPC": "D",
# 	"council_tax_band": "D",
# 	"lease": "999 years from 25/12/1972",
# 	"remaining_lease": 948
# }

In [ ]:
#@title Test data 2 (138. Carisbrooke Court, Cheam) { vertical-output: true }

print("""
https://www.rightmove.co.uk/properties/132170990#/?channel=RES_BUY
price: null
floor: null
sqft: null
parking: Garage en-bloc
service charge: £1,008 per annum - For the period 31st December 2022 to 30th December 2023. Charged bi-annually.
ground rent: null
chain: NO ONWARD CHAIN
EPC: D
Council tax: C
Lease: 999 years less 3 days from 10 May 1982
tenure Share of freehold
""")

data = """
Key features

    *Spacious apartment*Modern kitchen & bathroom*Two double bedrooms*Garage en-bloc*Secure phone entry system*Share of freehold*Offered with NO ONWARD CHAIN*Next to Cheam train station*Easy reach of local shops, schools & amenities*Buyers Commission May Be Required

Property description
PRIME LOCATION - Offered to the market with NO ONWARD CHAIN is this two double bedroom apartment complete with SECURE PHONE ENTRY SYSTEM and set in the heart of Cheam, just seconds away from Cheam train station providing fantastic transport links. The apartment is impressively SPACIOUS throughout providing plenty of storage space and comprises; Entrance hall, large bright and airy reception room, MODERN fully fitted kitchen and bathroom and two good sized bedrooms both with built in wardrobes. Outside the property boasts a garage en-bloc and access to lovely communal grounds. Cheam village is just a short walk away providing easy access to a selection of shops and amenities along with local schools including; Avenue Primary, Homefield, Nonsuch Girls and St Dunstan's making this the perfect location for all your everyday needs. This is a great opportunity that will not be around for long, so call now to arrange a viewing and avoid disappointment. EPC Rating D.

*Spacious apartment
*Modern kitchen & bathroom
*Two double bedrooms
*Garage en-bloc
*Secure phone entry system
*Share of freehold
*Offered with NO ONWARD CHAIN
*Next to Cheam train station
*Easy reach of local shops, schools & amenities
*Buyers Commission May Be Required

Entrance Hall

Doors to;

Reception Room

4.85m x 3.25m

Kitchen

3.15m x 1.9m

Bedroom One

4.27m x 2.77m

Bedroom Two

3.89m x 2.13m

Bathroom

Communal Grounds

Garage en-bloc

4.8m x 2.5m

Tenure

Share of Freehold - Term: 999 years less 3 days from 10 May 1982.

Council Tax Band

C.

Service Charge

£1,008 per annum - For the period 31st December 2022 to 30th December 2023. Charged bi-annually.

Shareholder Administration Fee

£30 per annum

Buyers Commission May Be Required*

Full details available upon request - *This property is being marketed by Choices on behalf of the seller on the basis that the buyer pays our fee of between 2.4% incl VAT and 3.6% incl VAT of the net purchase price. Unless otherwise agreed offers will therefore be submitted to the seller net of our fee.

NB

Please note photos were taken prior to tenants moving in.

Brochures
"""

In [ ]:
#@title Pulling data RightMove pages

import bs4
import requests
from bs4 import BeautifulSoup

URL = "https://www.rightmove.co.uk/properties/132170990#/?channel=RES_BUY" #@param {type:"string"}

r = requests.get(URL)
  
# soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install 
soup = BeautifulSoup(r.content, 'html.parser')


# Find the titl element
elements = soup.find_all("h1")
title = None
for e in elements:
  if e.get("itemprop") == "streetAddress":
    title = e.string

# Find the element that contains the text
elements = soup.find_all("article")
data_element = None
for e in elements:
  # print(e.get("data-testid"))
  # print(e.decode_contents())
  if e.get("data-testid") == "primary-layout":
    data_element = e
    break

perform_on_type(
    data_element,
    lambda x: isinstance(x, bs4.element.NavigableString), # matcher
    lambda x: x.string.replace_with(x.string + " ")) # fn

# Remove tooltips
for e in data_element.find_all("span"):
  if e.get("role") == "tooltip":
    # tooltip_elements.append(e)
    e.clear()

data = data_element.text
data = data.replace("\r\n", " ")
data = data.replace("\n", " ")
data = data.replace("\r", " ")

print(title)
# print(len(data))
print(data)

In [ ]:
#@title gpt-3.5-turbo { vertical-output: false }

# Prepare question

fields = [
  "floor",
  "sqft",
  "parking/garage",
  "service charge",
  "ground rent",
  "chain",
  "EPC",
  "council tax",
  "leasehold/freehold/tenure",
  "lease",
]
prompt_format_1 = """
A table summerizing the following data (put "null" if the information is not available):

{data}

| """ + " | ".join(fields) + " | "

question = prompt_format_1.format(data=data)

def format_output(output):
  values = output.split("|")
  if len(values) == len(fields) + 2:
    if values[0].strip() == "" and values[-1].strip() == "":
      values = values[1:-1]
  if len(values) != len(fields):
    print("Incorrect number of fields: %d (expect: %d)" % (len(values), len(fields)))
    print("Expected fields: %s" % "|".join(fields))
    return

  out_format = "%% %ds: %%s" % np.max([len(x) for x in fields])
  for i in range(len(fields)):
    print(out_format % (fields[i], values[i].strip()))


# Call ChatGPT API
response_3_5_turbo = openai.ChatCompletion.create(   
  model="gpt-3.5-turbo",
  temperature=0.2, # (0 to 2) low value = more deteriministic
  # frequency_penalty=-1.8, # (-2 to 2) penalty for repeating words verbatim, in this case we do want it to repeat verbatim
  messages=[         
    {
      "role": "user",
      "content": question
    },
  ])
response_string = response_3_5_turbo.choices[0].message.content
format_output(response_string)
print("Raw response: \"%s\"" % response_string)

In [ ]:
#@title davinci { vertical-output: false }

response_davinci = openai.Completion.create(
  model="text-davinci-003",
  prompt=question,
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response_json = response_davinci.choices[0].text
print(response_json)